In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import re

In [4]:
start = time.time()
# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
driver = webdriver.Chrome(ChromeDriverManager().install())

# si on a télécharger le driver directement dans le dossier 
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service)

# Navigate to the website
# driver.get("https://www.billboard.com/charts/hot-100/")
driver.get("https://www.billboard.com/charts/billboard-global-200/")
# driver.get("https://www.billboard.com/charts/billboard-200/")

# Wait for the page to load
# driver.implicitly_wait(10)
time.sleep(5)

# reject all cookies
driver.find_element(By.XPATH,"/html/body/div[5]/div[2]/div/div/div[2]/div/div/button[1]").click()

iterateur100 = list(range(1,110,11))
iterateur100.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

time.sleep(50)
# nb_max 111 ou 221
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]

allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

# for i, j in zip(allList, allXpath):
#     i = scraper(i,j,iterateur200,221)

df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts'])
print(df)

# Close the webdriver
driver.close()

end = time.time()
temps = end - start
print("temps d'execution :",temps,"s")

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 41.1MB/s]
C:\Users\thoma\AppData\Local\Temp\ipykernel_19768\1624214680.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


                            Title                        Artist Rank  \
0                         Flowers                   Miley Cyrus    1   
1                       Kill Bill                           SZA    2   
2    Bzrp Music Sessions, Vol. 53            Bizarrap & Shakira    3   
3                          Unholy        Sam Smith & Kim Petras    4   
4                       Calm Down           Rema & Selena Gomez    5   
..                            ...                           ...  ...   
195                    In The End                   Linkin Park  196   
196                       Ferrari  James Hype & Miggy Dela Rosa  197   
197          Just The Way You Are                    Bruno Mars  198   
198              Call Out My Name                    The Weeknd  199   
199                     Good Days                           SZA  200   

    Last Week Peak Positon Weeks on charts  
0           1            1               4  
1           3            1               9  


In [5]:
song_artists = list(zip(df['Artist'],df["Title"]))
song_artists

[('Miley Cyrus', 'Flowers'),
 ('SZA', 'Kill Bill'),
 ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'),
 ('Sam Smith & Kim Petras', 'Unholy'),
 ('Rema & Selena Gomez', 'Calm Down'),
 ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"),
 ('Harry Styles', 'As It Was'),
 ('RAYE Featuring 070 Shake', 'Escapism'),
 ('David Guetta & Bebe Rexha', "I'm Good (Blue)"),
 ('Taylor Swift', 'Anti-Hero'),
 ('Morgan Wallen', 'Last Night'),
 ('NewJeans', 'OMG'),
 ('Miguel', 'Sure Thing'),
 ('NewJeans', 'Ditto'),
 ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"),
 ('Yandel & Feid', 'Yandel 150'),
 ('Manuel Turizo', 'La Bachata'),
 ('Stephen Sanchez', 'Until I Found You'),
 ('JVKE', 'Golden Hour'),
 ('The Weeknd', 'Die For You'),
 ('Meghan Trainor', 'Made You Look'),
 ('Arcangel & Bad Bunny', 'La Jumpa'),
 ('Beyonce', 'Cuff It'),
 ('OneRepublic', "I Ain't Worried"),
 ('The Weeknd & Ariana Grande', 'Save Your Tears'),
 ('Tom Odell', 'Another Love'),
 ('Drake & 21 Savage', 'Rich Flex'),
 ('SZA',

In [6]:
# #modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):
    
    if 'X' in song_artists[i][0] and 'Lil Nas X' not in song_artists[i][0]:
        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("X", "&")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Fuerza Regida & Grupo Frontera', 'Bebe Dame')
('Peso Pluma & Natanael Cano', 'PRC')
('Carin Leon & Grupo Frontera', 'Que Vuelvas')
('Junior H & Oscar Maydon', 'Fin de Semana')
('Imagine Dragons & JID', 'Enemy')
('TOMORROW & TOGETHER', 'Sugar Rush Ride')
('Fuerza Regida & Natanael Cano', 'Ch y La Pizza')


In [7]:
for i in range(len(song_artists)):

    if 'vs.' in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("vs.", "")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

print(song_artists)

('Lana Del Rey  Cedric Gervais', 'Summertime Sadness')
[('Miley Cyrus', 'Flowers'), ('SZA', 'Kill Bill'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('Harry Styles', 'As It Was'), ('RAYE Featuring 070 Shake', 'Escapism'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Taylor Swift', 'Anti-Hero'), ('Morgan Wallen', 'Last Night'), ('NewJeans', 'OMG'), ('Miguel', 'Sure Thing'), ('NewJeans', 'Ditto'), ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('Stephen Sanchez', 'Until I Found You'), ('JVKE', 'Golden Hour'), ('The Weeknd', 'Die For You'), ('Meghan Trainor', 'Made You Look'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('Beyonce', 'Cuff It'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Tom Odell', 'Another Love'), ('Drake & 21 Sa

In [8]:
#modification du nom de l'artiste pour la recherche Genius

for i in range(len(song_artists)):

    if " With " in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace(" With ", " ")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

print(song_artists)

('Kane Brown Katelyn Brown', 'Thank God')
[('Miley Cyrus', 'Flowers'), ('SZA', 'Kill Bill'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('Harry Styles', 'As It Was'), ('RAYE Featuring 070 Shake', 'Escapism'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Taylor Swift', 'Anti-Hero'), ('Morgan Wallen', 'Last Night'), ('NewJeans', 'OMG'), ('Miguel', 'Sure Thing'), ('NewJeans', 'Ditto'), ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('Stephen Sanchez', 'Until I Found You'), ('JVKE', 'Golden Hour'), ('The Weeknd', 'Die For You'), ('Meghan Trainor', 'Made You Look'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('Beyonce', 'Cuff It'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Tom Odell', 'Another Love'), ('Drake & 21 Savage', 'Rich 

In [9]:
#modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):

    if "Coolio Featuring L.V." in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("Coolio Featuring L.V.", "Colio")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Colio', "Gangsta's Paradise")


In [10]:
#modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):

    if "Quevedo & Mike Towers" in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("Quevedo & Mike Towers", "Quevedo")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Quevedo', 'Playa del Ingles')


In [11]:
song_artists.remove(('Carin Leon & Grupo Frontera', 'Que Vuelvas'))
print(('Carin Leon & Grupo Frontera', 'Que Vuelvas') in song_artists)

False


In [87]:
driver_genius = webdriver.Chrome(ChromeDriverManager().install())

    # ouvrir le site et accepter les cookies

driver_genius.get("https://genius.com/")
driver_genius.implicitly_wait(10)
driver_genius.find_element(By.ID,"onetrust-accept-btn-handler").click()
time.sleep(3)
 
song,artist,date,producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags = [],[],[],[],[],[],[],[]


for i in range(len(song_artists)):

    driver_genius.get("https://genius.com/")

    # activer la recherche
    element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
    element.send_keys(song_artists[i][1]+' '+song_artists[i][0], Keys.ENTER)
    driver_genius.implicitly_wait(10)
    song.append(song_artists[i][1])
    artist.append(song_artists[i][0])
    driver_genius.implicitly_wait(10)
    # clicker sur la chanson
    try :
        
        driver_genius.implicitly_wait(10)
        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()
        driver_genius.implicitly_wait(10)
    
    except:

        driver_genius.implicitly_wait(10)
        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a").click()
        driver_genius.implicitly_wait(10)

    driver_genius.implicitly_wait(10)

    #scroll to credits

    target_credits = driver_genius.find_element(By.CSS_SELECTOR,"div.SongInfo__Title-nekw6x-1")
    driver_genius.execute_script("arguments[0].scrollIntoView();", target_credits)

    driver_genius.implicitly_wait(10)


    # Si le bouton expand est présent

    try: 
        #click on expand if this button exist

        all_elements_after_target_credit = target_credits.find_elements(By.XPATH, ".//following::*")
        driver_genius.implicitly_wait(10)
        # trouve le 1er element dont le tag_name est "button"

        for element in all_elements_after_target_credit:
            if element.tag_name == "button" and "Expand" in element.text:
                element.click()
                break

        driver_genius.implicitly_wait(10)
    
    except:

        # Le bouton Expand n'a pas été trouvé sur la page
        print("Le bouton n'existe pas sur la page")

    
    try:

        # Localiser la balise "div" contenant la date en utilisant son contenu texte
        date_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Release Date')]")
        driver_genius.implicitly_wait(10)
        # Extraire la date à partir du texte en utilisant des expressions régulières
        if date_element:
            date_text = date_element.find_element(By.XPATH,'./following-sibling::div').text
            date_regex = r"\w+\s+\d{1,2},\s+\d{4}"
            time.sleep(2)
            match = re.search(date_regex, date_text)
            if match:

                date.append(match.group())
        
        driver_genius.implicitly_wait(10)

    except:

        date.append("Nan")

    driver_genius.implicitly_wait(10)

    #Recherche des producteurs,ecrivains,publisher,distributor et tags

    ##################################################PRODUCTEURS#############################################
    try:
        producteurs_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Produced By')]")
        producteurs_text.append(producteurs_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        producteurs_text.append("Nan")
    ############################################ECRIVAIN#############################################
    driver_genius.implicitly_wait(10)
    try:
        ecrivains_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Written By')]")
        ecrivains_text.append(ecrivains_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:  
        ecrivains_text.append("Nan")
    ########################################PUBLISHER##################################################
    driver_genius.implicitly_wait(10)
    try:
        publisher_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Publisher')]")
        publisher_text.append(publisher_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        publisher_text.append("Nan")
    #######################################DISTIBUTEURS###############################################
    driver_genius.implicitly_wait(10)
    try:
        distributor_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Distributor')]")
        distributor_text.append(distributor_element.find_element(By.XPATH,'./following-sibling::div').text)
    except:
        distributor_text.append("Nan")

    #############################################TAGS#####################################################
    driver_genius.implicitly_wait(10)

    try:
        tag_pos = driver_genius.find_element(By.CLASS_NAME,"SongTags__Title-xixwg3-0")
        driver_genius.execute_script("arguments[0].scrollIntoView();", tag_pos)
        name = driver_genius.find_elements(By.CLASS_NAME, "SongTags__Container-xixwg3-1")

        driver_genius.implicitly_wait(10)

        tag=[]

        for elt in name:

            tag.append(elt.text)

        tags = ''.join(set(tag))  
        
        Tags.append(tags.replace('\n',', '))
        driver_genius.implicitly_wait(10)

    except:

        Tags.append("Nan")
        driver_genius.implicitly_wait(10)
    # on revient à l'accueil
    
        driver_genius.get("https://genius.com/")
    # verif = (date[i],song[i])
    #print(date[i],song[i])
    print(i)
    driver_genius.implicitly_wait(10)
    
print(date, producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags)
# df_genius = pd.DataFrame( {'Realise Date': date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text, 'Tags':distributor_text})

C:\Users\thoma\AppData\Local\Temp\ipykernel_19768\3487647624.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_genius = webdriver.Chrome(ChromeDriverManager().install())


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Le bouton n'existe pas sur la page
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Le bouton n'existe pas sur la page
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
Le bouton n'existe pas sur la page
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
Le bouton n'existe pas sur la page
90
91
92
93
94
95
96
97
98
99
100
101
Le bouton n'existe pas sur la page
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
Le bouton n'existe pas sur la page
126
127
128
129
130
131
Le bouton n'existe pas sur la page
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
Le bouton n'existe pas sur la page
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
['January 13, 2023', 'December 9, 

In [88]:
print(len(date))
print(len(producteurs_text))
print(len(ecrivains_text))
print(len(publisher_text))
print(len(distributor_text))
print(len(Tags))

199
199
199
199
199
199


In [89]:
print(date)

['January 13, 2023', 'December 9, 2022', 'January 11, 2023', 'September 22, 2022', 'August 26, 2022', 'December 2, 2022', 'April 1, 2022', 'October 12, 2022', 'July 31, 2012', 'October 21, 2022', 'January 31, 2023', 'January 2, 2023', 'November 30, 2010', 'December 19, 2022', 'February 3, 2023', 'December 20, 2022', 'May 26, 2022', 'September 1, 2021', 'July 15, 2022', 'November 25, 2016', 'October 21, 2022', 'November 30, 2022', 'July 29, 2022', 'May 13, 2022', 'April 23, 2021', 'October 15, 2012', 'November 4, 2022', 'December 9, 2022', 'December 16, 2022', 'February 2, 2023', 'October 7, 2022', 'October 4, 2019', 'October 17, 2022', 'November 30, 2022', 'July 6, 2022', 'August 5, 2022', 'November 26, 2016', 'January 27, 2023', 'November 24, 2022', 'July 17, 2022', 'November 3, 2022', 'May 6, 2022', 'April 22, 2022', 'June 29, 2022', 'January 24, 2023', 'May 6, 2022', 'December 8, 2022', 'July 9, 2021', 'November 29, 2019', 'June 29, 2020', 'October 28, 2022', 'September 21, 2016', '

In [66]:
for i in range(len(date)):

    if date[i]=="Nan":

        print(i)

In [70]:
print(len(song))

78


In [91]:
df_genius_200 = pd.DataFrame( { 'song':song, 'artist':artist, 'date':date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text,'Tags':Tags})
df_genius_200

,song,artist,date,Producers,Writes,Publisher,Distributors,Tags
0,Flowers,Miley Cyrus,"January 13, 2023",Kid Harpoon & Tyler Johnson,"Miley Cyrus, Aldae & Michael Pollack","Pure Tone Music, Suga Bear Recordz, Warner-Tam...",Nan,"Pop, Funk-Pop, Dance Rock, Pop-Rock, Nu Disco,..."
1,Kill Bill,SZA,"December 9, 2022",Carter Lang & Rob Bisel,"SZA, Rob Bisel & Carter Lang",Nan,Sony Music Entertainment,"Pop, Alternative R&B, Alternative Pop, Psyched..."
2,"Bzrp Music Sessions, Vol. 53",Bizarrap & Shakira,"January 11, 2023",Bizarrap,"Shakira, KEITYN, Santiago Alvarado & Bizarrap",Nan,The Orchard,"Pop, Deep House, Synth-Pop, Latin Urban, Latin..."
3,Unholy,Sam Smith & Kim Petras,"September 22, 2022","Blake Slatkin, Cirkut, ILYA, Jimmy Napes, Omer...","Blake Slatkin, Cirkut, ILYA, Jimmy Napes, Kim ...",Nan,Nan,"R&B, Pop, UK Pop, Electronic, TikTok, Dark Pop..."
4,Calm Down,Rema & Selena Gomez,"August 26, 2022",Andrevibez,"Andrevibez, Rema, Michael Hunter & Selena Gomez",Nan,Universal Music Group,"Pop, Remix, Nigerian Pidgin, Nigeria, Afrobeats"
...,...,...,...,...,...,...,...,...
194,In The End,Linkin Park,"October 9, 2001",Don Gilmore,"Mr. Hahn, Rob Bourdon, Mike Shinoda, Chester B...",Nan,Nan,"Rap, Rock, Hard Rock, Alternative Metal, Rap M..."
195,Ferrari,James Hype & Miggy Dela Rosa,"March 15, 2022",James Hype,"James Hype, Miggy Dela Rosa, Josh Grimmett, Jo...",Nan,Nan,"Pop, Deep House, House, Alternative Dance, Ele..."
196,Just The Way You Are,Bruno Mars,"July 20, 2010",Needlz & The Smeezingtons,"Philip Lawrence, Ari Levine, Needlz, Saint Cas...",Nan,Nan,"R&B, Pop, Alternative R&B, Piano, Ballad, Adul..."
197,Call Out My Name,The Weeknd,"March 30, 2018",Ging,"Ging, Nicolas Jaar & The Weeknd","Alma Road Music, 4AD Music, Beggars Music, Win...",Nan,"Pop, R&B, Singer-Songwriter, Downtempo, Canada..."


In [92]:
df_genius_200.to_csv('my_data.csv', index=False)